In [4]:
from bs4 import BeautifulSoup as bs
from datetime import datetime
from datetime import date
from datetime import timedelta
from fractions import Fraction
import requests
import cloudscraper
import pandas as pd
import math

## Takes in url and outputs dataframe into csv with prices

In [39]:
today = date.today()
tomorrow = today + timedelta(days=1)
dayafternext = tomorrow + timedelta(days=1)
str_today = str(today)
str_tomorrow = str(tomorrow)
str_dayafternext = str(dayafternext)

def per_chance(d):
    d = str(Fraction(d)+1)
    return round((1/d)*100,2)


def scrape(url):

    scraper = cloudscraper.create_scraper(browser={'browser': 'firefox','platform': 'windows','mobile': False})

    r = scraper.get(url).content


    # Convert to bs object
    soup = bs(r,'lxml')
    contents = soup.prettify()

    bad_bookies = ["888", "Mansion", "Betfred", "Betway"]

    class_tags = ["bc bs oo b",
                        "bc bs oi",
                        "bc bs o",
                        "bc bs oo",
                        "bc bs o",
                        "bc bs oo win-only",
                        "bc bs o b",
                        "bc bs oi b",
                       ]


    bookies = ["b365",
                   "SkyBet",
                   "PaddyPower",
                   "William Hill",
                   "888",
                   "Betfair",
                   "BetVictor",
                   "Coral / Ladbrokes",
                   "Unibet",
                   "Mansion",
                   "Betfred",
                   "Betway",
                   "Boyles"
        ]

    names = soup.find_all('a', class_="popup selTxt")
    all_names = []

        # Eliminating Non Runners
    nr = "(N/R)"

        # Appending each name to all_names list
    i=0
    while i < len(names):
        if nr in names[i]:
            i+=1
        else:
            all_names.append(names[i].text) 
            i+=1

        # Create dataframe & add names
    df = pd.DataFrame({'Name':all_names})

        # Cleaning
    df['Name'] = df['Name']
    for column in df.columns:
        df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)


        # Adding columns
    df = df.assign(**dict.fromkeys(bookies,0))
    df[bookies] = df[bookies].astype("float")

    all_data = soup.find('tr', class_="diff-row evTabRow bc")
    all_prices = all_data.find_all('td',class_=class_tags)
    all_data = soup.find_all('tr', class_="diff-row evTabRow bc")



    num_of_horses = len(df)
    k = 0 
    horses = []

    while k< num_of_horses:
        name = "horse" + str(k)
        name = all_data[k].find_all('td',class_=class_tags)
        horses.append(name)
        k+=1


    horse_number = 0
    i = 0 
    SP = "SP"

    while horse_number < num_of_horses:
        i=0
        select_prices = []
        while i < len(bookies):
            select_prices.append(horses[horse_number][i].text)     
            df[bookies[i]][horse_number] = select_prices[i]  
            i+=1

        horse_number+=1





    df = df.drop(columns = bad_bookies)
    df["365%"] = per_chance(df["b365"])
    df["PP%"] = per_chance(df["PaddyPower"])

    df["AVG%"] = round(((per_chance(df["SkyBet"]) + per_chance(df["PaddyPower"]) + per_chance(df["William Hill"]) + 
                         per_chance(df["Betfair"]) + per_chance(df["BetVictor"]) + per_chance(df["Coral / Ladbrokes"]) +
                         per_chance(df["Unibet"]) + per_chance(df["Boyles"]))/8),2)

    df["% Diff (365 v PP)"] = df["PP%"] - df["365%"]
    df["% Diff (365 v AVG)"] = df["AVG%"] - df["365%"]


    peep = 0
    sheet_num = str(peep)
    race_info = soup.find('div', class_="left-if-has-race-info")
    race_info = race_info.text
    meeting = race_info.split()[0]
    time_of_race = race_info.split()[2]
    which_race = meeting
    df.to_excel("HorseRacing_"+str_today+".xlsx", sheet_name = "sheet_"+sheet_num, index=False)

In [43]:
def per_chance(d):
    d = float(Fraction(d)+1)
    return round((1/d)*100,2)


In [46]:
per_chance("34")

2.86

In [40]:
scrape("https://www.oddschecker.com/horse-racing/chelmsford-city/17:00/winner")

<ipython-input-39-f9c67745344d>:106: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[bookies[i]][horse_number] = select_prices[i]


TypeError: argument should be a string or a Rational instance

### Notes
- make several dfs for each race and then merge all?